In [1]:
import torch as pt
import pandas as pd
import numpy as np
import pyproj as pp
import tqdm
import gc
import optuna
import atd2025

In [2]:
# pt.set_default_device("gpu:0")

In [3]:
# Read dataset in
data = pd.read_csv("https://www.maserv.work/ATD/model2/ucf_atd_model/datasets/dataset1_truth.csv")
data["time"] = pd.to_datetime(data["time"])

In [4]:
wgs84 = pp.CRS.from_epsg(4326)
utm = pp.CRS.from_epsg(32616)
to_utm = pp.Transformer.from_crs(wgs84, utm)

rad_earth = 6371000

x, y = to_utm.transform(data["lat"].to_numpy(), data["lon"].to_numpy())

x = x.astype("float32")
y = y.astype("float32")

# Read in rest of data
t = data["time"]
t = (t - t[0]).astype("timedelta64[s]").astype("int").astype("float32")
speed = data["speed"].to_numpy().astype("float32")
course = data["course"].to_numpy().astype("float32") * np.pi / 180

x = pt.tensor(x)
y = pt.tensor(y)
t = pt.tensor(t)
dx = pt.tensor(np.sin(course) * speed)
dy = pt.tensor(np.cos(course) * speed)

gc.collect()

20

In [5]:
def objective(trial: optuna.Trial):
    max_size = data.shape[0]
    state_size = 5

    # Keeps all tracks we have previously found
    mus = pt.zeros((max_size, state_size, 1))
    covs = pt.zeros((max_size, state_size, state_size))
    last_times = pt.zeros((max_size))
    track_id = pt.zeros((max_size), dtype=pt.long)

    # Used to keep track of where we are when filtering
    idxs = pt.arange(max_size)

    noise_x = trial.suggest_float("nx", 0.1, 1000, log=True)
    noise_dx = trial.suggest_float("ndx", 0.01, 10, log=True)
    noise_y = trial.suggest_float("ny", 0.1, 1000, log=True)
    noise_dy = trial.suggest_float("ndy", 0.01, 10, log=True)
    omega_int = trial.suggest_float("omega", -3.1415926536, 3.1415926536)

    meas_noise = pt.diag(pt.tensor([noise_x, noise_dx, noise_y, noise_dy], dtype=pt.float32))
    H = pt.zeros((state_size - 1, state_size))
    H[:, :state_size - 1] = pt.eye(state_size - 1)

    # Noise propogation constants
    qw = trial.suggest_float("qw", 1e-8, 1, log=True) 
    qx = trial.suggest_float("qx", 1e-6, 1, log=True)
    qy = trial.suggest_float("qy", 1e-6, 1, log=True)

    cut_dist = trial.suggest_float("cut", 1e-6, 100, log=True)

    # Prior state covariance matrix
    prior_cov = (pt.eye(state_size) * 0.001)
    prior_cov[:state_size - 1, :state_size - 1] += meas_noise

    # Generate state transition matrix for CTRV
    def F_gen(dt, filter, next_track_id):
        omega = mus[:next_track_id][filter][:, 4, 0]
        F = pt.zeros((pt.sum(filter), state_size, state_size))
        
        sinw = pt.sin(omega * dt)
        cosw = pt.cos(omega * dt)

        # Set constants
        F[:, 0, 0] = 1
        F[:, 2, 2] = 1
        F[:, 4, 4] = 1

        # Set already calculated
        F[:, 1, 1] = cosw
        F[:, 1, 3] = -1 * sinw
        F[:, 3, 1] = sinw
        F[:, 3, 3] = sinw

        # Set new calculations
        toSet = sinw / omega
        F[:, 0, 1] = toSet
        F[:, 2, 3] = toSet

        toSet = (1 - cosw) / omega
        F[:, 0, 3] = -1 * toSet
        F[:, 2, 1] = toSet

        return F

    # Generate state noise matrix for CTRV
    def Q_gen(dt, filter):
        Q = pt.zeros((pt.sum(filter), state_size, state_size))
        dt3 = (dt ** 3) / 3
        dt2 = (dt ** 2) / 2
        
        Q[:, 0, 0] = qx * dt3
        Q[:, 0, 1] = qx * dt2
        Q[:, 1, 0] = qx * dt2
        Q[:, 1, 1] = qx * dt

        Q[:, 2, 2] = qy * dt3
        Q[:, 2, 3] = qy * dt2
        Q[:, 3, 2] = qy * dt2
        Q[:, 3, 3] = qy * dt

        Q[:, 4, 4] = qw * dt

        return Q

    # Add new track to list
    def add_track(next_track_id, i):
        # Create state
        ti = t[i]
        xi = x[i]
        yi = y[i]
        dxi = dx[i]
        dyi = dy[i]

        # Update
        track_id[i] = next_track_id
        mus[next_track_id] = pt.tensor([xi, dxi, yi, dyi, omega_int + 1e-15])[:, None]
        covs[next_track_id] = prior_cov
        last_times[next_track_id] = ti
        
    # Update an old track in the list
    def update_track(best_id, mu, cov, S, y_resid, time, i):
        # Calculate Kalman gain / intermediate value
        K = (pt.linalg.solve(S, H) @ cov).T
        imk = pt.eye(state_size) - (K @ H)
        
        # Calculate posterior mu/cov
        post_mu = mu + (K @ y_resid) 
        post_cov = (imk @ cov @ imk.T) + (K @ meas_noise @ K.T)

        # Update
        track_id[i] = best_id
        mus[best_id] = post_mu
        covs[best_id] = post_cov
        last_times[best_id] = time

    # Actual algorithm
    next_track_id = 0
    for i, time in tqdm.tqdm(enumerate(t), total=t.shape[0]):
        if next_track_id == 0:
            add_track(next_track_id, i)
            next_track_id += 1
            continue

        time_filter = (time - last_times[:next_track_id]) >= 2
        if not pt.any(time_filter):
            add_track(next_track_id, i)
            next_track_id += 1
            continue
        else:
            # Check best manhalobis distance, update if below threshold
            prev_times = last_times[:next_track_id][time_filter]
            dt = time - prev_times

            filtered_idxs = idxs[:next_track_id][time_filter]
            est_cov = covs[:next_track_id][time_filter]
            est_mu = mus[:next_track_id][time_filter]

            # Get F and Q
            F = F_gen(dt, time_filter, next_track_id)
            Q = Q_gen(dt, time_filter)

            # Get prior guess of state
            est_mu = F @ est_mu
            est_cov = (F @ est_cov @ F.mT) + Q

            # Get prior guess of measurement
            est_mu_y = H @ est_mu
            est_cov_y = (H @ est_cov @ H.T) + meas_noise

            # Get man dist between measurement and guess for measurement
            measurement = pt.tensor([x[i], dx[i], y[i], dy[i]])[:, None]
            measurement_resid = measurement - est_mu_y
            man_dist = (measurement_resid.mT @ pt.linalg.solve(est_cov_y, measurement_resid)).squeeze()

            best_man_dist_idx = pt.argmin(man_dist)
            best_man_dist = man_dist[best_man_dist_idx]
            best_track_id = track_id[filtered_idxs[best_man_dist_idx]]
            if best_man_dist <= cut_dist:
                # Update track
                mu = est_mu[best_man_dist_idx]
                cov = est_cov[best_man_dist_idx]
                S = est_cov_y[best_man_dist_idx]
                y_resid = measurement_resid[best_man_dist_idx]
                update_track(best_track_id, mu, cov, S, y_resid, time, i)
            else:
                # New track
                add_track(next_track_id, i)
                next_track_id += 1
                continue

    tids = track_id.numpy()
    pids = data["point_id"]

    pd.DataFrame({"point_id": pids, "track_id": tids}).to_csv("grademe.csv")
    gc.collect()
    return atd2025.accuracy.evaluate_predictions("grademe.csv", "https://www.maserv.work/ATD/model2/ucf_atd_model/datasets/dataset1_truth.csv")

In [6]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2025-10-04 05:39:04,772] A new study created in memory with name: no-name-d806a33a-4115-40aa-8c13-f8f23a8794e7
100%|██████████| 102861/102861 [07:16<00:00, 235.40it/s]
[I 2025-10-04 05:46:36,504] Trial 0 finished with value: 0.10477245992164183 and parameters: {'nx': 6.119967046280383, 'ndx': 0.017356680784218582, 'ny': 11.664097443522317, 'ndy': 7.271949540490696, 'omega': -2.3187104061059536, 'qw': 0.0009182370596619432, 'qx': 5.02782293857852e-06, 'qy': 1.5561520219403863e-05, 'cut': 0.019298400700966115}. Best is trial 0 with value: 0.10477245992164183.
100%|██████████| 102861/102861 [07:31<00:00, 227.92it/s]
[I 2025-10-04 05:54:26,219] Trial 1 finished with value: 0.06037273602239916 and parameters: {'nx': 289.6196787102076, 'ndx': 0.09852341795229083, 'ny': 7.7930466591229175, 'ndy': 0.029109381271910935, 'omega': -0.520509935649828, 'qw': 0.0004382379664424167, 'qx': 0.007005886992475888, 'qy': 3.972390652951933e-05, 'cut': 7.859114826673729e-05}. Best is trial 0 with value: 